In [ ]:
# importing the required libraries
import os
import numpy as np
import pandas as pd
import time

import matplotlib.pyplot as plt
import cv2
import imageio
import imgaug as ia
import imgaug.augmenters as iaa

import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

from PIL import Image
import glob
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import torch
from torchvision import datasets, models, transforms
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import random_split

In [ ]:
!pip install torch
!pip install torchvision

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
ERROR: Operation cancelled by user
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
ERROR: Operation cancelled by user


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
input_path = "/content/drive/MyDrive/Dataset0"

In [ ]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

In [ ]:
data_transforms = {
    'Dataset0':
    transforms.Compose([
        transforms.Resize((224,224)),
        #Dataaugmentation
        #transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)),
        ##transforms.RandomRotation(10),
        ##transforms.RandomErasing(scale=(0.02, 0.16), ratio=(0.3, 1.6)),
        #transforms.RandomPerspective(distortion_scale=0.2),
        #transforms.RandomHorizontalFlip(),
        ##transforms.CenterCrop(10),
        transforms.ToTensor(),
        normalize
    ]),
    #'TE_PARKTRAIN':
    #transforms.Compose([
    #    transforms.Resize((224,224)),
    #    transforms.ToTensor(),
    #    normalize
    #]),
}
image_datasets = {
    'Dataset0':
    datasets.ImageFolder(input_path + '', data_transforms['Dataset0']),
    #'TE_PARKTRAIN':
    #datasets.ImageFolder(input_path + 'TE_PARKTRAIN', data_transforms['TE_PARKTRAIN'])
}

# Define the ratio for splitting the data (e.g., 80% for training, 20% for testing)
train_ratio = 0.8
test_ratio = 1 - train_ratio

# Split the datasets into training and testing sets
train_size_A = int(train_ratio * len(image_datasets['Dataset0']))
test_size_A = len(image_datasets['Dataset0']) - train_size_A

train_dataset_A, test_dataset_A = random_split(image_datasets['Dataset0'], [train_size_A, test_size_A])

train_dataloader_A = torch.utils.data.DataLoader(train_dataset_A, batch_size=5, shuffle=True, num_workers=0)
test_dataloader_A = torch.utils.data.DataLoader(test_dataset_A, batch_size=5, shuffle=False, num_workers=0)

In [ ]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
#model = models.inception_v3(pretrained=True).to(device)
#model.fc = nn.Linear(in_features=2048, out_features=2, bias=True).to(device) #adapt last layer
#model.fc = nn.Linear(in_features=2048, out_features=2, bias=True).to(device) #adapt last layer
model = models.mobilenet_v2(pretrained=True)
print(model)
model.classifier = torch.nn.Sequential(
    torch.nn.Dropout(p=0.2, inplace=False),
    torch.nn.Linear(1280, 2)
)
model = model.to(device)
print(model)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth
100%|██████████| 13.6M/13.6M [00:00<00:00, 119MB/s]


MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

In [ ]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.005, weight_decay = 0.005, momentum = 0.9)

In [ ]:
corrects=0
num_epochs=50
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_dataloader_A):
        # Move tensors to the configured device
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        #outputs= outputs.logits
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print ('Epoch [{}/{}], Step [{}], Loss: {:.4f}'
                   .format(epoch+1, num_epochs, i+1, loss.item()))
    tp = 0
    fp = 0
    fn = 0
    # Validation
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_dataloader_A:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            tp += ((predicted == labels) & (predicted == 1)).sum().item()
            fp += ((predicted != labels) & (predicted == 1)).sum().item()
            fn += ((predicted != labels) & (predicted == 0)).sum().item()

            del images, labels, outputs

        print('Accuracy of the network on the validation images: {} %'.format(100 * correct / total))
        # Calculate precision, recall, and F1 score
        precision = tp / (tp + fp)
        recall = tp / (tp + fn)
        f1 = 2 * (precision * recall) / (precision + recall)

        print('Precision: {:.4f}'.format(precision))
        print('Recall: {:.4f}'.format(recall))
        print('F1 Score: {:.4f}'.format(f1))
    if corrects<=correct:
      corrects=correct
      torch.save(model.state_dict(), '/content/drive/MyDrive/models0/CNN.h5')

Epoch [1/50], Step [862], Loss: 0.6941
Accuracy of the network on the validation images: 52.319109461966605 %


ZeroDivisionError: division by zero

Feature Extraction

In [ ]:
import torch
import torchvision.transforms as transforms
from torchvision import models
from PIL import Image

# Preprocess images
preprocess = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load the pre-trained MobileNetV2 model
model = models.mobilenet_v2(pretrained=True)
model.classifier = torch.nn.Sequential(
    torch.nn.Dropout(p=0.2, inplace=False),
    torch.nn.Linear(1280, 2)
)
# Load the saved model weights
model.load_state_dict(torch.load("/content/drive/MyDrive/models0/CNN.h5", map_location=torch.device('cpu')))

# Use the features before the classifier layer to extract embeddings
layer = model.features

# Define the function to get feature vectors
def get_vector(image_path):
    # Load and preprocess the image
    image = Image.open(image_path)
    image = preprocess(image).unsqueeze(0)

    # Create a tensor to store the features
    my_embedding = torch.zeros(1280)

    # Define a function to copy the output of the layer
    def copy_data(m, i, o):
        my_embedding.copy_(torch.mean(o.squeeze(), dim=[1,2])) # Global Average Pooling

    # Attach the copy function to the selected layer
    h = layer.register_forward_hook(copy_data)

    # Pass the image through the model
    with torch.no_grad():
        model(image)

    # Detach the copy function from the layer
    h.remove()

    # Return the feature vector
    return my_embedding

# Test the get_vector function with an image path
image_path = "/content/drive/MyDrive/EssentialTremorProject/Normal/patient32/0-crops/32-tes-0-1-R-crops/1.jpg"
vector = get_vector(image_path)
print(vector.numpy())


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth
100%|██████████| 13.6M/13.6M [00:00<00:00, 54.7MB/s]


[0.21662383 0.21546754 0.22965686 ... 0.09036835 0.15718202 0.18949118]


In [ ]:
import csv
import os

# Définir le chemin pour enregistrer les vecteurs dans le fichier CSV
output_vectors_Train = "/content/drive/MyDrive/Data0/vecteursTrain.csv"

# Définir le chemin pour enregistrer les labels dans le fichier CSV
output_labels_Train = "/content/drive/MyDrive/Data0/labelsTrain.csv"

with open(output_vectors_Train, 'w', newline='') as vectors_csvfile, open(output_labels_Train, 'w', newline='') as labels_csvfile:
    vectors_writer = csv.writer(vectors_csvfile)
    labels_writer = csv.writer(labels_csvfile)

    for class_dir in ["Healthy0", "TE0"]:
        listPatient = os.listdir("/content/drive/MyDrive/PFA/"+class_dir+"/0")

        for patient in listPatient:
            listVideos = os.listdir("/content/drive/MyDrive/PFA/"+class_dir+"/0/"+patient)
            if (class_dir == 'Healthy0' and patient == '77') or (class_dir == 'TE0' and patient == '36'):
                exit()
            else:
                listVideost = os.listdir("/content/drive/MyDrive/PFA/"+class_dir+"/0/"+patient)

                for video in listVideos:
                    if os.path.isdir("/content/drive/MyDrive/PFA/"+class_dir+"/0/"+patient+"/"+video):
                        if os.path.exists("/content/drive/MyDrive/PFA/"+class_dir+"/0/"+patient+"/"+video+"/"):
                            path = "/content/drive/MyDrive/PFA/"+class_dir+"/0/"+patient+"/"+video+"/"
                        else:
                            path = "/content/drive/MyDrive/PFA/"+class_dir+"/0/"+patient+"/"+video+"/"

                        listFrames = os.listdir(path)

                        i = 9
                        nbrFrame = 0

                        for frame in listFrames:
                            if i % 2 == 0 and nbrFrame < 25:
                                nbrFrame += 1
                                vector = get_vector(path + frame)
                                vectors_writer.writerow(vector.numpy())

                                if class_dir == "Healthy0":
                                    labels_writer.writerow([0])
                                elif class_dir == "TE0":
                                    labels_writer.writerow([1])

                            i += 1


In [ ]:
import os
listPatient = os.listdir("/content/drive/MyDrive/PFA/Healthy/90")
listPatient

['23',
 '02',
 '82',
 '81',
 '80',
 '79',
 '78',
 '77',
 '76',
 '75',
 '74',
 '73',
 '69',
 '68',
 '63',
 '59',
 '58 (1)',
 '58',
 '57',
 '56',
 '55',
 '54']

In [ ]:
import os
listPatient = os.listdir("/content/drive/MyDrive/PFA/TE0/0")
listPatient

['00',
 '31',
 '07',
 '08',
 '12',
 '09',
 '14',
 '15',
 '16',
 '18',
 '20',
 '23',
 '24',
 '30',
 '32',
 '35',
 '36',
 '41',
 '42',
 '114',
 '115',
 '139']

In [ ]:
import csv

# Définir le chemin pour enregistrer les vecteurs dans le fichier CSV
output_vectors_Train = "/content/drive/MyDrive/Data0/vecteursTest.csv"

# Définir le chemin pour enregistrer les labels dans le fichier CSV
output_labels_Train = "/content/drive/MyDrive/Data0/labelsTest.csv"
with open(output_vectors_Train, 'w', newline='') as vectors_csvfile, open(output_labels_Train, 'w', newline='') as labels_csvfile:
  vectors_writer = csv.writer(vectors_csvfile)
  labels_writer = csv.writer(labels_csvfile)
  for class_dir in ["Healthy0", "TE0"]:
    listPatient=os.listdir("/content/drive/MyDrive/PFA/"+class_dir+"/0")
    print(listPatient)
    for patient in listPatient:
      if (class_dir== 'Healthy0' and (patient == '58' or patient == '57' or patient == '56' or patient == '55' or patient == '54')) or (class_dir== 'TE0' and (patient == '41' or patient == '42' or patient == '114' or patient == '115' or patient == '139')):
        listVideost=os.listdir("/content/drive/MyDrive/PFA/"+class_dir+"/0/"+patient)
        print(listVideost)
        for video in listVideost:
          os.listdir("/content/drive/MyDrive/PFA/"+class_dir+"/0/"+patient+"/"+video)
          listframes=os.listdir("/content/drive/MyDrive/PFA/"+class_dir+"/0/"+patient+"/"+video)
          path="/content/drive/MyDrive/PFA/"+class_dir+"/0/"+patient+"/"+video+"/"
        else:

          listframes=os.listdir("/content/drive/MyDrive/PFA/"+class_dir+"/0/"+patient+"/"+video)
          path="/content/drive/MyDrive/PFA/"+class_dir+"/0/"+patient+"/"+video
          i=9
          nbrFrame=0

          for frame in listframes:
            i=i+1
            if i%2==0 and nbrFrame<25:
              nbrFrame=nbrFrame+1
              vector=get_vector(path+"/"+frame)
              # Écrire le vecteur dans le fichier CSV des vecteurs
              vectors_writer.writerow(vector.numpy())
              # Écrire le label dans le fichier CSV des labels
              if class_dir== "Healthy0":
                labels_writer.writerow([0])
              if class_dir=="TE0":
                labels_writer.writerow([1])

['23', '02', '82', '81', '80', '79', '78', '77', '76', '75', '74', '73', '69', '68', '63', '59', '58 (1)', '58', '57', '56', '55', '54']
['58T-0-11-R-crops', '58T-0-2-L-crops', '58T-0-10-R-crops', '58T-0-6-R-crops', '58T-0-5-L-crops', '58T-0-9-R-crops', '58T-0-4-L-crops', '58T-0-8-R-crops', '58T-0-3-L-crops', '58T-0-7-R-crops', '58T-0-1-L-crops']
['57T-0-8-R-crops', '57T-0-5-L-crops', '57T-0-4-L-crops', '57T-0-11-R-crops', '57T-0-9-R-crops', '57T-0-10-R-crops', '57T-0-6-L-crops', '57T-0-3-L-crops', '57T-0-7-R-crops', '57T-0-12-R-crops', '57T-0-1-L-crops', '57T-0-2-L-crops']
['56T-0-3-L-crops', '56T-0-5-L-crops', '56T-0-10-R-crops', '56T-0-12-R-crops', '56T-0-8-R-crops', '56T-0-9-R-crops', '56T-0-7-R-crops', '56T-0-6-L-crops', '56T-0-11-R-crops', '56T-0-4-L-crops', '56T-0-1-L-crops', '56T-0-2-L-crops']
['55T-0-9-R-crops', '55T-0-8-R-crops', '55T-0-11-R-crops', '55T-0-10-R-crops', '55T-0-3-L-crops', '55T-0-7-R-crops', '55T-0-4-L-crops', '55T-0-6-L-crops', '55T-0-12-R-crops', '55T-0-5-L-c

**LSTM**

In [ ]:
import numpy as np
from numpy import genfromtxt
from keras.models import Sequential
from keras.layers import Dense, LSTM, BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.regularizers import l2
from time import time

def convert_to_floats(arr):
    result = map(float, arr)
    return list(result)

dataTraining = genfromtxt("/content/drive/MyDrive/Data0/vecteursTrain.csv", delimiter=',', dtype=float)
LabelTraining = genfromtxt("/content/drive/MyDrive/Data0/labelsTrain.csv", delimiter=',', dtype=float)
dataTesting = genfromtxt("/content/drive/MyDrive/Data0/vecteursTest.csv", delimiter=',', dtype=float)
Labelcsv_testLabels = genfromtxt("/content/drive/MyDrive/Data0/labelsTest.csv", delimiter=',', dtype=float)

scaled_train_features = dataTraining
train_labels = LabelTraining
T = 25

X_train, y_train = [], []
for i in range(0, train_labels.shape[0], 25):
    X_train.append(scaled_train_features[i:i+T])
    y_train.append(train_labels[i])

max_length = max(len(seq) for seq in X_train)
X_train_padded = np.array([np.pad(seq, ((0, max_length - len(seq)), (0, 0)), mode='constant') for seq in X_train])

y_train = np.array(y_train).reshape(-1, 1)
print(f'Train data dimensions: {len(X_train_padded)}, {y_train.shape}')

test_labels = Labelcsv_testLabels
scaled_test_features = dataTesting

X_test, y_test = [], []
for i in range(0, test_labels.shape[0], 25):
    X_test.append(scaled_test_features[i:i+T])
    y_test.append(test_labels[i])

X_test_padded = np.array([np.pad(seq, ((0, max_length - len(seq)), (0, 0)), mode='constant') for seq in X_test])
y_test = np.array(y_test).reshape(-1, 1)
print(f'Test data dimensions: {X_test_padded.shape}, {y_test.shape}')

LAYERS = [256, 128, 64, 1]
M_TRAIN = X_train_padded.shape[0]
M_TEST = X_test_padded.shape[0]
N = X_train_padded.shape[2]
BATCH = M_TRAIN
EPOCH = 100
LR = 5e-2
LAMBD = 3e-2
DP = 0.0
RDP = 0.0

print(f'layers={LAYERS}, train_examples={M_TRAIN}, test_examples={M_TEST}')
print(f'batch = {BATCH}, timesteps = {T}, features = {N}, epochs = {EPOCH}')
print(f'lr = {LR}, lambda = {LAMBD}, dropout = {DP}, recurr_dropout = {RDP}')

model = Sequential()
model.add(LSTM(input_shape=(max_length, N), units=LAYERS[0],
               activation='tanh', recurrent_activation='hard_sigmoid',
               kernel_regularizer=l2(LAMBD), recurrent_regularizer=l2(LAMBD),
               dropout=DP, recurrent_dropout=RDP,
               return_sequences=True, return_state=False,
               stateful=False, unroll=False
              ))
model.add(BatchNormalization())
model.add(LSTM(units=LAYERS[1],
               activation='tanh', recurrent_activation='hard_sigmoid',
               kernel_regularizer=l2(LAMBD), recurrent_regularizer=l2(LAMBD),
               dropout=DP, recurrent_dropout=RDP,
               return_sequences=True, return_state=False,
               stateful=False, unroll=False
              ))
model.add(BatchNormalization())
model.add(LSTM(units=LAYERS[2],
               activation='tanh', recurrent_activation='hard_sigmoid',
               kernel_regularizer=l2(LAMBD), recurrent_regularizer=l2(LAMBD),
               dropout=DP, recurrent_dropout=RDP,
               return_sequences=False, return_state=False,
               stateful=False, unroll=False
              ))
model.add(BatchNormalization())
model.add(Dense(units=LAYERS[3], activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              metrics=['accuracy'],
              optimizer=Adam(learning_rate=LR))
print(model.summary())

lr_decay = ReduceLROnPlateau(monitor='loss',
                             patience=1, verbose=0,
                             factor=0.5, min_lr=1e-8)

early_stop = EarlyStopping(monitor='val_accuracy', min_delta=0,
                           patience=30, verbose=1, mode='auto',
                           baseline=0, restore_best_weights=True)

start = time()
History = model.fit(X_train_padded, y_train,
                    epochs=EPOCH,
                    batch_size=BATCH,
                    validation_split=0.0,
                    validation_data=(X_test_padded[:M_TEST], y_test[:M_TEST]),
                    shuffle=True,verbose=0,
                    callbacks=[lr_decay, early_stop])
print('-'*65)
print(f'Training was completed in {time() - start:.2f} secs')
print('-'*65)

train_loss, train_acc = model.evaluate(X_train_padded, y_train,
                                       batch_size=M_TRAIN, verbose=0)
test_loss, test_acc = model.evaluate(X_test_padded[:M_TEST], y_test[:M_TEST],
                                     batch_size=M_TEST, verbose=0)
print('-'*65)
print(f'train accuracy = {round(train_acc * 100, 4)}%')
print(f'test accuracy = {round(test_acc * 100, 4)}%')
print(f'test error = {round((1 - test_acc) * M_TEST)} out of {M_TEST} examples')

# Calculate precision, recall, and F1-score on the test set
y_pred = model.predict(X_test_padded[:M_TEST])
y_pred_binary = (y_pred > 0.5).astype(int)  # Convert probabilities to binary predictions

precision = precision_score(y_test[:M_TEST], y_pred_binary)
recall = recall_score(y_test[:M_TEST], y_pred_binary)
f1 = f1_score(y_test[:M_TEST], y_pred_binary)

print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1-score: {f1}')

Train data dimensions: 395, (395, 1)
Test data dimensions: (10, 25, 1280), (10, 1)
layers=[256, 128, 64, 1], train_examples=395, test_examples=10
batch = 395, timesteps = 25, features = 1280, epochs = 100
lr = 0.05, lambda = 0.03, dropout = 0.0, recurr_dropout = 0.0


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 25, 256)           1573888   
                                                                 
 batch_normalization (Batch  (None, 25, 256)           1024      
 Normalization)                                                  
                                                                 
 lstm_1 (LSTM)               (None, 25, 128)           197120    
                                                                 
 batch_normalization_1 (Bat  (None, 25, 128)           512       
 chNormalization)                                                
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 batch_normalization_2 (Bat  (None, 64)                2

## Tester une vidèo

In [ ]:
import csv
import os

# Définir le chemin pour enregistrer les vecteurs dans le fichier CSV
output_vectors_Train = "/content/drive/MyDrive/Data0/vecteursTest.csv"
output_labels_Train = "/content/drive/MyDrive/Data0/labelsTest.csv"  # Définir le chemin pour enregistrer les labels

with open(output_vectors_Train, 'w', newline='') as vectors_csvfile, open(output_labels_Train, 'w', newline='') as labels_csvfile:
    vectors_writer = csv.writer(vectors_csvfile)
    # Votre code pour lire les images et écrire les vecteurs dans le fichier CSV



    path = "/content/drive/MyDrive/EssentialTremorProject/Normal/patient20/0-crops/20-tes-0-1-R-crops"
    listFrames = os.listdir(path)
    i = 9
    nbrFrame = 0
    for frame in listFrames:
      if i % 2 == 0 and nbrFrame < 25:
        nbrFrame += 1
        vector = get_vector(path +"/"+ frame)
        vectors_writer.writerow(vector.numpy())
      i += 1

In [ ]:
!pip install tensorflow

In [ ]:
import numpy as np
from numpy import genfromtxt
import torch
from torchvision.models import mobilenet_v2
from time import time

# Charger les données de test à partir du fichier CSV
dataTesting1 = genfromtxt("/content/drive/MyDrive/Data0/vecteursTest.csv", delimiter=',', dtype=float)
T = 25
scaled_test_features1 = dataTesting1

X_test1 = []
for i in scaled_test_features1:
    X_test1.append(i)

# Remplir les données de test pour atteindre une longueur de 25 (si nécessaire)
X_test_padded1 = np.array([np.pad(X_test1, ((0, 25 - len(X_test1)), (0, 0)), mode='constant')])

# Ajouter deux canaux supplémentaires pour correspondre aux trois canaux attendus par MobileNetV2
X_test_tensor = torch.tensor(X_test_padded1, dtype=torch.float32)
X_test_tensor = X_test_tensor.unsqueeze(1).repeat(1, 3, 1, 1)

# Charger le modèle PyTorch MobileNetV2 pré-entraîné
model = mobilenet_v2(pretrained=True)

# Mettre le modèle en mode évaluation
model.eval()

# Faire une prédiction avec le modèle
with torch.no_grad():
    y_pred_group1 = model(X_test_tensor)

# Convertir les prédictions en classes binaires
y_pred_binary_group = (y_pred_group1 > 0.5).int()
classe = y_pred_binary_group[0][0].item()  # Convertir le tenseur en scalaire Python

print("Classe de la vidéo :", classe)


Classe de la vidéo : 0
